## Improt Libraries and packages

In [1]:
# json related
import json
import pandas as pd
from jsonpath import jsonpath
# db related
import sqlite3
from sqlite3 import Error
from string import Template
import math

import datetime
import time

from ipywidgets import widgets
from IPython.display import display, clear_output,HTML

import numpy as np
import pandas as pd
from functools import reduce

from wordcloud import WordCloud, STOPWORDS
from matplotlib import pyplot as plt

import matplotlib.pyplot as plt
import plotly as py
import plotly.express  as px 

## Init

In [2]:
#创建连接
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn
#创建数据表
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
#判断视频是否存在
def noexist_video(conn,data):
    with conn:
        sql="SELECT id From video where video_id =?"
        cur = conn.cursor()
        cur.execute(sql,[data])
        result = cur.fetchone()
        return False if result else True

#插入数据
def addVideo(conn,data):
    with conn:
        res={'video_id':'','info_id':'','text':''}
        if(noexist_video(conn,data['video_id'])):
            sql_video = "INSERT INTO  video(id,video_id,title,publishedAt,channelId,channelTitle,categoryID,thumbnail_link,comments_disabled,ratings_disabled,description) SELECT MAX(id)+1, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?  FROM video"
            cur_video = conn.cursor()
            cur_video.execute(sql_video, (data['video_id'],data['title'],data['publishedAt'],data['channelId'],
                              data['channelTitle'],data['categoryID'],data['thumbnail_link'],data['comments_disabled'],
                              data['ratings_disabled'],data['description']))
            video_id=cur_video.lastrowid
            res['video_id']=video_id
        sql_video_log = "INSERT INTO  video_log(id,video_id,trending_date,tags,view_count,likes,dislikes,comment_count) SELECT MAX(id)+1,?, ?, ?, ?, ?, ?, ?  FROM video_log"
        cur_video_log = conn.cursor()
        cur_video_log.execute(sql_video_log, (data['video_id'],data['trending_date'],data['tags'],data['view_count'],data['likes'],data['dislikes'],data['comment_count']))
        info_id=cur_video_log.lastrowid
        res['info_id']=info_id
        res['info']="success"if  info_id>0 else "faled"
        return res
    
#删除数据
def delete_video(conn, video_id):
    sql_video = "DELETE FROM video WHERE video_id= ?"
    cur_video = conn.cursor()
    cur_video.execute(sql_video,[video_id])
    sql_video_log = "DELETE FROM video_log WHERE video_id= ?"
    cur_video_log = conn.cursor()
    cur_video_log.execute(sql_video_log,[video_id])
    conn.commit()
    print('DELETE SUCCESS')

def selet_video(conn,data):
    with conn:
        data=str(data) if isinstance(data,tuple) else "('"+str(data)+"')"
        sql_l="SELECT * FROM video where video_id in "+data
        cur = conn.cursor()
        cur.execute(sql_l)
        return cur.fetchall()
#查找模板
def video_num(conn):
    cur = conn.cursor()
    cur.execute("select count(*) from video")
    total=cur.fetchone()[0]
    return {'total': total, 'pagesize':math.ceil(total/15) }
def selet_stl(conn,type):
    with conn:
        sql_l="SELECT * FROM  stl   where type in(0,?)"
        cur = conn.cursor()
        cur.execute(sql_l,[type])
        return cur.fetchall()
    
def list_videos(conn,page):
    with conn:
        if(video_num(conn)['pagesize']<page):
            return 'Maximum number of pages exceeded'
        else:
            sql_l=Template("select * from video ORDER By id limit (${this_page}-1)*15,15").substitute(this_page=page)
            return pd.read_sql(sql_l, conn)

def videos_trend(conn,video_id):
    with conn:
        sql_l=Template("select b.id,b.video_id,b.title,b.publishedAt,b.channelId,b.channelTitle,b.categoryId,b.thumbnail_link,b.comments_disabled,b.ratings_disabled,b.description,a.trending_date,a.tags,a.view_count,a.likes,a.dislikes,a.comment_count from video_log as a,video as b where a.video_id=b.video_id  and a.video_id='${this_video}' ORDER By b.id").substitute(this_video=video_id)
        return pd.read_sql(sql_l,conn)
    
#插入文章
def insertArticle(conn,data):
    with conn:
        res=[]
        cur = conn.cursor()
        cur.execute("SELECT id From article where video_id =?",[data['video_id']])
        count=cur.fetchone()
        if(count):
            sql_article = "UPDATE article SET title=?,content=?,data=?,type=?,click=?,createtime=? where video_id= ?"
            cur_article = conn.cursor()
            cur_article.execute(sql_article,(data['title'],data['content'],data['data'],data['type'],data['click'],data['createtime'],data['video_id']))
            res.append(cur_article.lastrowid)
        else:
            sql_article = "INSERT INTO  article(id,title,content,data,type,click,video_id,createtime) SELECT MAX(id)+1, ?, ?, ?, ?, ?, ?, ?  FROM article"
            cur_article = conn.cursor()
            cur_article.execute(sql_article, (data['title'],data['data'],data['content'],data['type'],data['click'],data['video_id'], data['createtime']))
            res.append(cur_article.lastrowid)
        return res
def cate_info(conn,data):
        cate_infos = Template("select * from video where categoryID = ${cateId}").substitute(cateId=data)
        res=pd.read_sql(cate_infos, conn)
        return res

In [3]:
def see_video(conn,show_v):
    num=video_num(conn)
    page=int(input("please input the page to view the article,the total num is "+str(num['total'])+" and the maxpage is "+str(num['pagesize'])+' \n '))
    if(num['pagesize']<page):
        return "max pageSize is ",num['pagesize']
    if(show_v):
        display(list_videos(conn,page))
    else:
        return list_videos(conn,page)
    
#数据格式 做线图
def get_attr_video(pd_temp):
    df_con=pd.DataFrame({'id':[],'video_id':[],'trending_date':[],'tags':[],'value':[]})
    for index,key in enumerate(pd_temp.loc[::,['view_count','likes','dislikes','comment_count']]):
        res=pd_temp.loc[::,['id','video_id','tags','trending_date',key]]
        res.columns=['id','video_id','tags','trending_date','value']
        res['type']=[key for i in range(len(pd_temp))]
        df_con=df_con.append(res,ignore_index=True)
    return df_con

In [4]:
def youtube_init(dbConnection):
    #创建分类表
    sql_create_category_table = """ CREATE TABLE IF NOT EXISTS category (
                                        cid integer PRIMARY KEY,
                                        kind text NOT NULL,
                                        etag text,
                                        title text,
                                        assignable text,
                                        channelid text
                                     ); """
    #创建视频记录表
    sql_create_video_log_table = """ CREATE TABLE IF NOT EXISTS video_log (
                                          id integer PRIMARY KEY,
                                          video_id text,
                                          trending_date text,
                                          tags text,
                                          view_count text,
                                          likes text,
                                          dislikes text,
                                          comment_count text
                                     ); """
    #创建视频表
    sql_create_video_table=""" CREATE TABLE IF NOT EXISTS video (
                                          id integer PRIMARY KEY,
                                          video_id text,
                                          title text,
                                          publishedAt text,
                                          channelId text,
                                          channelTitle text,
                                          categoryId text,
                                          thumbnail_link text,
                                          comments_disabled text,
                                          ratings_disabled text,
                                          description text
                                     ); """
    #创建文章表
    sql_create_article_table = """ CREATE TABLE IF NOT EXISTS article (
                                        id integer PRIMARY KEY,
                                        title text NOT NULL,
                                        content text,
                                        type text,
                                        data text
                                        type text,
                                        click integer,
                                        video_id text,
                                        createtime text
                                     ); """
    #创建模板库
    sql_create_stl_table = """ CREATE TABLE IF NOT EXISTS stl (
                                        id integer PRIMARY KEY,
                                        pre_title text NOT NULL,
                                        type text,
                                        used text,
                                        createtime text
                                     ); """
    #创建分类表
    create_table(dbConnection,sql_create_category_table)
    #导入并创建视频信息表
    create_table(dbConnection,sql_create_video_table)
    create_table(dbConnection,sql_create_video_log_table)
    #创建模板库表和文章库表
    create_table(dbConnection,sql_create_stl_table)
    create_table(dbConnection,sql_create_article_table)
    
    #从json读取数据
    with open('archive/US_category_id.json') as json_data:
        data_detail = json.load(json_data)
        kind=jsonpath(data_detail,"$..kind")
        etag=jsonpath(data_detail,"$..etag")
        ids=jsonpath(data_detail,"$..id")
        title=jsonpath(data_detail,"$..title")
        assignable=jsonpath(data_detail,"$..assignable")
        channelId=jsonpath(data_detail,"$..channelId")
        json_data=pd.DataFrame({'kind':kind,'etag':etag,'cid':ids,'title':title,'assignable':assignable,'channelId':channelId})
    #导入数据到category
    json_data.to_sql('category', con=dbConnection, if_exists='append', index=None)
    #从表格读取数据
    df_info=pd.read_csv('./archive/US_youtube_trending_data.csv')
    df_video=df_info[['video_id','title','publishedAt','channelId','channelTitle','categoryId','thumbnail_link','comments_disabled','ratings_disabled','description',]].copy()
    video_log=df_info[['video_id', 'trending_date','tags', 'view_count','likes','dislikes','comment_count']].copy()
    df_video['id']=[i+1 for i in range(len(df_video))]
    video_log['id']=[j+1 for j in range(len(video_log))]
    df_video=df_video.drop_duplicates(subset=['video_id'],keep='first')
    #导入数据到video和video_log
    df_video.to_sql('video', con=dbConnection, if_exists='append', index=None)
    video_log.to_sql('video_log', con=dbConnection, if_exists='append', index=None)

In [5]:
def common_Template(df_temp):
    res={'last_date':'',
         'pre_date':'',
         'min_play':'',
         'max_play':'',
         'play':'',
         'min_likes':'',
         'max_likes':'',
         'likes':'',
         'min_dislikes':'',
         'max_dislikes':'',
         'dislikes':'',
         'min_comment_count':'',
         'max_comment_count':'',
         'min_likes':'',
         'max_likes':''
        }
    res['last_date']=df_temp['trending_date'].min()
    res['pre_date']=df_temp['trending_date'].max()
    res['min_play']=df_temp['view_count'].min()
    res['max_play']=df_temp['view_count'].max()
    res['play']=int(res['max_play'])-int(res['min_play'])
    res['min_likes']=df_temp['likes'].min()
    res['max_likes']=df_temp['likes'].max()
    res['likes']=int(res['max_likes'])-int(res['min_likes'])
    
    res['min_dislikes']=df_temp['dislikes'].min()
    res['max_dislikes']=df_temp['dislikes'].max()
    res['dislikes']=int(res['max_dislikes'])-int(res['min_dislikes'])
    
    res['min_comment_count']=df_temp['comment_count'].min()
    res['max_comment_count']=df_temp['comment_count'].max()
    res['comment_count']=int(res['max_comment_count'])-int(res['min_comment_count'])
    
    res['min_likes']=df_temp['likes'].min()
    res['max_likes']=df_temp['likes'].max()
    
    days=datetime.datetime.strptime(res['pre_date'], '%Y-%m-%d')-datetime.datetime.strptime(res['last_date'], '%Y-%m-%d')
    res['days'] = int(days.days)
    return  res

In [6]:
def show_articles(conn,page):
    with conn:
        if(article_num(conn)['pagesize']<page):
            return 'Maximum number of pages exceeded'
        else:
            sql_l=Template("select * from article ORDER By id limit (${this_page}-1)*15,15").substitute(this_page=page)
            pf=pd.read_sql(sql_l, conn)
            display(pd.read_sql(sql_l, conn))
            str_l=""
            #for row in pf.itertuples():
                #div
                 #getattr(row, 'c1'), getattr(row, 'c2')) 
            
#查找模板
def article_num(conn):
    cur = conn.cursor()
    cur.execute("select count(*) from article")
    total=cur.fetchone()[0]
    return {'total': total,'pagesize':math.ceil(total/15)}

In [7]:
#show_articles(dbConnection,0)

In [8]:
def cate_clouImage(cate):
    df=cate_info(dbConnection,cate)
    fig,ax=plt.subplots(figsize=(16,8))
    text=','.join(list(df['title']))
    remove_w=['Video','Got','do','it']+ list(STOPWORDS)
    my_mask=plt.imread("bg.jpg")
    mycloudword=WordCloud(width=300, 
                          height=150, 
                          scale=1, 
                          margin=2,
                          mask=my_mask,
                          background_color='#373E4b',
                          max_words=200, 
                          min_font_size=30, 
                          max_font_size=90,
                          stopwords=remove_w,
                          random_state=60).generate(text)

    ax.imshow(mycloudword)
    ax.axis("off")
    plt.savefig("images/"+str(round(time.time()*1000))+".jpg")
    plt.show()

In [9]:
#连接数据库
file = "./archive/youtube.db"
dbConnection = create_connection(file)
#youtube_init(dbConnection)

##  Main function to  operate video data

In [10]:
def showOptions():
    while True:
        """ Show the options """
        choices = ["add a video", "delet a video", "view video informations","exit"]
        print("---------------------------")
        print("Please choose following option:")
        print("1. ", choices[0])
        print("2. ", choices[1])
        print("3. ", choices[2])
        print("4. ", choices[3])
        choice = input('Please enter your option: ')
        # with conn:
        if choice == '4':
            break
        elif choice == '1':
            clear_output()
            info={"video_id":'',"title":'',"publishedAt":'',"channelId":'',"channelTitle":'',"categoryID":'',"thumbnail_link":'',"comments_disabled":'',"ratings_disabled":'',"description":'',"trending_date":'',"tags":'',"view_count":'',"likes":'',"dislikes":'',"comment_count":''}
            for i in info:
                info[i]=input('Please enter the video :'+i+'\n')
            res=addVideo(dbConnection,info)
            print(res['info'])
        elif choice == '2':
            clear_output()
            video_id=input('Please enter the videoid you want to delete: ')
            delete_video(dbConnection,video_id)
        elif choice == '3':
            clear_output()
            see_video(dbConnection,True)
        else:
            print("This option is not available")
            continue

## Main function to create article

In [15]:
def create_articles(dbConnection):
        def handle_submit(attrs):
            with output:
                clear_output()
                res=selet_video(dbConnection,(attrs.new))[0]
                attrs=['id','video_id','title','publishedAt','channelId','channelTitle','categoryId','thumbnail_link', 'comments_disabled', 'ratings_disabled','description']
                final_json=dict(zip(attrs,res))
                #print(final_json)
                filename='d.json'
                with open(filename,'w') as file_obj:
                     json.dump(final_json,file_obj)
                #print(final_json)
                df_in=videos_trend(dbConnection,final_json['video_id'])
                df_in['trending_date']=df_in['trending_date'].str[0:10]

                final_json={**final_json,**common_Template(df_in)}
                temp_list=selet_stl(dbConnection,int(final_json['categoryId']))
                temp_str='';
                for temp in temp_list:
                    temp_str+=Template(temp[1]).substitute(final_json)
                content="<p class='content'>"+temp_str+"</p>"
                date_this=datetime.date.today()
                today=str(date_this.year)+'-'+str(date_this.month)+'-'+str(date_this.day)
                data_list=get_attr_video(df_in)
                fig=px.line(data_list, 
                       x="trending_date",
                       y="value",
                        color="type"
                      )
                style="""
                <style>
                    .my_span{margin-right: 8px;padding: 0 4px;font-size: 16px;}
                    .tags{background: #f2f2f2;color: rgba(0, 0, 0, 0.3);font-size: 16px;margin-right: 8px;}
                    .author{color: #576b95;font-size: 16px;margin-right: 8px;}
                    .createtime{color: #999;font-size: 16px;margin-right: 8px;}
                    .title{font-size:30px;color: #576b95;font-weight:bold;margin-left:15px}
                    .title_article{color: rgb(53, 179, 120);font-size: 23px;margin-top: 1em; margin-bottom: 0rem; padding-top: 0.5em; padding-bottom: 0.5em;font-weight: bold;}
                    .content{font-size: 16px;padding-top: 8px;padding-bottom: 8px;text-decoration: none;line-height: 26px;color: #bbb;margin: 1em 4px;}
                </style>
                """
                title="<h1> Video-"+final_json['title']+" analysis-"+today+"</h1>"
                tag="<span class='my_span tags'>Original</span>"
                author="<span  class='author'>Dandelion</span>"
                createtime="<span  class='createtime'>"+today+"</span>"
                tags="""
                <div style="width: 100%;border-radius: 15px;background: #2c305c;margin-top:15px;
                padding:15px;
                margin-bottom:15px">

                <div class="article-tag-card__left">
                    <div class="title">Tages</div>
                    <div class="article-tag__item-wrp no-active js_tag">
                     <span class="article-tag__item">
                    """+data_list['tags'][0]+"""
                </span>
                </div>
                </div>
                </div>
                """
                disct_art={'title':final_json['title'],'content':temp_str,'data':str(final_json),'type':0,'click':1, 'createtime':str(round(time.time()*1000)),'video_id':final_json['video_id']}
                insertArticle(dbConnection,disct_art)
                video_link="<a target='_blank' href='https://www.youtube.com/watch?v="+final_json['video_id']+"'><img src=\'https://i.ytimg.com/vi/"+final_json['video_id']+"/hqdefault.jpg\' width='100%'></a>"
                s=HTML(style+title+tag+author+createtime +tags+video_link+content )
                display(s)        
                display(HTML("""
                <div class="title_article">Line Chart</div>
                       """))
                fig.show()
                display(HTML("""
                <div class="title_article">Cloud Image</div>
                       """))
                cate_clouImage(final_json['categoryId'])
                display(HTML("<hr>"))

        lst=see_video(dbConnection,False)
        new_list=[]
        for key,value in enumerate(list(lst['title'])):
            new_list.append((lst['title'][key],lst['video_id'][key]))
        text = widgets.Dropdown(options=new_list,description='please select the article:')
        display(text)
        output = widgets.Output()
        display(output)
        text.observe(handle_submit, names='value')